# Возьмем готовую разбивку номенклатур по категориям

In [5]:
import pandas
import numpy as np
import codecs
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

colnames = ['name', 'id', 'class']
data = pandas.read_csv('nameCatIdCatName.csv', names=colnames, encoding='utf8')
names = data['name'].tolist()
classes = data['class'].tolist()
testLen = len(names) / 7
x_train = np.array(names[testLen:])
y_train = np.array(classes[testLen:])
x_test = np.array(names[:testLen]) 
y_test = np.array(classes[:testLen])

# Добавим две(sic!) строчки кода

In [3]:
pipeline = Pipeline([('vectorizer', TfidfVectorizer(ngram_range=(1,1))),('classifier',  KNeighborsClassifier())])
pipeline.fit(x_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_id...owski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'))])

In [4]:
score = pipeline.score(x_test, y_test)
print(score)

0.836408668731


# Полученная точность 84%! Это на 3% процента больше чем у комитета решающих деревьев и на 10% больше чем у нейронной сети! Хотя у леса точность чуть ниже, работает он намного быстрее, поэтому будем использовать его тоже

In [6]:
pipeline1 = Pipeline([('vectorizer', TfidfVectorizer(ngram_range=(1,1))),('classifier',  RandomForestClassifier())])
pipeline1.fit(x_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_id...imators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))])

In [7]:
score1 = pipeline1.score(x_test, y_test)
print(score1)

0.81213622291


# Раз уж мы тут все собрались, то посмотрим, что все это значит для наших  номенклатур

In [13]:
content = codecs.open("nom3utf.csv", 'r', 'utf8')
real_test = np.random.choice(np.array(content.readlines()), size = 11)

# Ближайшие соседи:

In [14]:
predicts = pipeline.predict(real_test)
for x, p in zip(real_test, predicts):
    print(x + "  " + p)

Вата стерильн. 250г СЦ МЕДТЕХНИКА ГУРМЭ
  ПЕРВАЯ ПОМОЩЬ
Диара капс. 2мг бл. N10 Оболенское ФП
  ЖЕЛУДОЧНО-КИШЕЧНЫЕ ПРЕПАРАТЫ
Обеграсс пор. 6.38г пак. N30 ACTAFARMA
  БИОЛОГИЧЕСКИ-АКТИВНЫЕ ДОБАВКИ
СПА БЕЛЛЬ Божья коровка грелка с наполнител. из гречих. и лаванд. Yangzhou King\sway Import & Export
  УХОД ЗА ТЕЛОМ
Ноопепт табл. 10мг бл. N50 Фармстандарт-Лексредства
  ЛЕКАРСТВА ПО РЕЦЕПТАМ
МЕРРИС трусики-подгузники XXL(15-28кг) N26 Kao Corp.
  МАМА И МАЛЫШ
ПАРОДОНТОЛ Зелен. чай паста зубная 124г Свобода
  УХОД ЗА ПОЛОСТЬЮ РТА
Р-Р Д/ЛИНЗ RENU MPS 240МЛ
  ЗАБОТА О ЗРЕНИИ
Дентипур порошок д/фиксации зубн.протезов 25г .@
  УХОД ЗА ПОЛОСТЬЮ РТА
Глюкозамин Максимум табл. N60 Herkel
  БИОЛОГИЧЕСКИ-АКТИВНЫЕ ДОБАВКИ
Deep depil крем-депилятор д/лица с масл персик 50мл [Ф126] Флоресан
  ГИГИЕНА


# Решающий лес:

In [16]:
predicts1 = pipeline1.predict(real_test)
for x, p in zip(real_test, predicts1):
    print(x + "  " + p)

Вата стерильн. 250г СЦ МЕДТЕХНИКА ГУРМЭ
  ПЕРВАЯ ПОМОЩЬ
Диара капс. 2мг бл. N10 Оболенское ФП
  ЛЕКАРСТВА ПО РЕЦЕПТАМ
Обеграсс пор. 6.38г пак. N30 ACTAFARMA
  БИОЛОГИЧЕСКИ-АКТИВНЫЕ ДОБАВКИ
СПА БЕЛЛЬ Божья коровка грелка с наполнител. из гречих. и лаванд. Yangzhou King\sway Import & Export
  УХОД ЗА БОЛЬНЫМИ
Ноопепт табл. 10мг бл. N50 Фармстандарт-Лексредства
  БИОЛОГИЧЕСКИ-АКТИВНЫЕ ДОБАВКИ
МЕРРИС трусики-подгузники XXL(15-28кг) N26 Kao Corp.
  МАМА И МАЛЫШ
ПАРОДОНТОЛ Зелен. чай паста зубная 124г Свобода
  УХОД ЗА ПОЛОСТЬЮ РТА
Р-Р Д/ЛИНЗ RENU MPS 240МЛ
  ЗАБОТА О ЗРЕНИИ
Дентипур порошок д/фиксации зубн.протезов 25г .@
  УХОД ЗА ПОЛОСТЬЮ РТА
Глюкозамин Максимум табл. N60 Herkel
  БИОЛОГИЧЕСКИ-АКТИВНЫЕ ДОБАВКИ
Deep depil крем-депилятор д/лица с масл персик 50мл [Ф126] Флоресан
  УХОД ЗА ЛИЦОМ


# Ну, не так уж и плохо